<a href="https://colab.research.google.com/github/TaisRol/ProcesamientoHabla/blob/main/Desafio_grupo_noticias_ROLDAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP3 - Procesamiento del habla
*Roldan Tais María Salomé*

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[1]

"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks."

In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez ajustado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 8754
print(newsgroups_train.data[idx])


/(hudson)
/If someone inflicts pain on themselves, whether they enjoy it or not, they
/are hurting themselves.  They may be permanently damaging their body.

That is true.  It is also none of your business.  

Some people may also reason that by reading the bible and being a Xtian
you are permanently damaging your brain.  By your logic, it would be OK
for them to come into your home, take away your bible, and send you off
to "re-education camps" to save your mind from ruin.  Are you ready for
that?  





/(hudson)
/And why is there nothing wrong with it?  Because you say so?  Who gave you
/the authority to say that, and set the standard for morality?

Why?

Because: 
I am a living, thinking person able to make choices for myself.
I do not "need" you to show me what you think is the way; I have observed
too many errors in your thinking already to trust you to make up the
rules for me.

Because:
I set the standard for my *own* morality, and I permit you to do 
the same for yourself.  I

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
cossim

array([0.1382319 , 0.1067036 , 0.23029327, ..., 0.12320753, 0.08765353,
       0.04415046])

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.22350593, 0.20131202, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 3000,  8097, 10836, ...,   426,  3559,  3521])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
mostsim

array([ 8097, 10836,  8726,  8754, 10229])

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.mideast'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

comp.sys.ibm.pc.hardware
alt.atheism
talk.politics.mideast
talk.religion.misc
talk.religion.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío


**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**No puedes usar la misma solución ya presentada por alguien en el foro antes que Ud. Es decir, sus 5 documentos al azar deben ser diferentes a los ya presentados, o las palabras que elija para el ejercicio 3 deben ser diferentes a las ya presentadas.**



In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from sklearn.model_selection import ParameterGrid

In [2]:
#carga de datos
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [8]:
#Vectorizamos el texto con TF-IDF
# Instanciamos un vectorizador
tfidfvect = TfidfVectorizer(stop_words='english', max_df=0.8, min_df=5)

# Transformamos directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data) # Matriz documento-término

# Vectorizamos los textos del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)

# Guardamos los targets que son enteros (etiquetas numéricas de clases)
y_train = newsgroups_train.target

#Y también los targets del conjunto de test
y_test = newsgroups_test.target


* Elegir los 5 documentos de manera aleatoria

In [9]:
np.random.seed(55)
documentos= np.random.choice(X_train.shape[0], size=5, replace=False)
print(documentos)

[6627 6824 6792  701 5769]


In [12]:
documentos = [6627, 6824, 6792, 701, 5769]

print("--- Etiquetas de los Documentos de Referencia ---")
for doc_id in documentos:
    etiqueta_numerica = newsgroups_train.target[doc_id]
    etiqueta_nombre = newsgroups_train.target_names[etiqueta_numerica]
    print(f"ID {doc_id}: {etiqueta_nombre}")

--- Etiquetas de los Documentos de Referencia ---
ID 6627: rec.motorcycles
ID 6824: sci.space
ID 6792: talk.politics.guns
ID 701: rec.motorcycles
ID 5769: rec.autos


###Similaridad de los documentos

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Índices seleccionados
documentos = [6627, 6824, 6792, 701, 5769]

# Itera sobre cada documento seleccionado
for idx_doc_referencia in documentos:
    # Obtiene el vector TF-IDF del documento de referencia
    vector_doc_ref = X_train[idx_doc_referencia]

    # Calcula la similaridad del coseno con todos los documentos
    # El resultado es un array 1D
    similaridades = cosine_similarity(vector_doc_ref, X_train).flatten()

    # Ordena los índices de similaridad de mayor a menor (descendente)
    # Excluye el primer índice (el documento consigo mismo)
    # [::-1] invierte el orden de los índices para que sea descendente
    indices_ranking = np.argsort(similaridades)[::-1]

    # Filtra los 5 documentos más similares, excluyendo el de origen (el primer elemento)
    top_5_indices = indices_ranking[1:6] # Tomamos del índice 1 al 5

    print(f"--- Documento de Referencia ID: {idx_doc_referencia} ---")
    print(f"Etiqueta: {newsgroups_train.target_names[newsgroups_train.target[idx_doc_referencia]]}")
    print("\nTop 5 Documentos Más Similares:")

    # Recupera y analiza los 5 documentos más similares
    for i, idx_similar in enumerate(top_5_indices):
        score = similaridades[idx_similar]
        etiqueta_similar = newsgroups_train.target_names[newsgroups_train.target[idx_similar]]

        # Muestra la información
        print(f"{i+1}. ID: {idx_similar} | Sim. Coseno: {score:.4f} | Etiqueta: {etiqueta_similar}")
    print("="*60, "\n")

        # Opcional: Mostrar fragmento de texto para el análisis
        # print(f"   Texto: {newsgroups_train.data[idx_similar][:100]}...")

--- Documento de Referencia ID: 6627 ---
Etiqueta: rec.motorcycles

Top 5 Documentos Más Similares:
1. ID: 8555 | Sim. Coseno: 0.2389 | Etiqueta: rec.autos
2. ID: 8266 | Sim. Coseno: 0.2380 | Etiqueta: rec.autos
3. ID: 2326 | Sim. Coseno: 0.2310 | Etiqueta: comp.windows.x
4. ID: 1401 | Sim. Coseno: 0.2308 | Etiqueta: rec.autos
5. ID: 5507 | Sim. Coseno: 0.2303 | Etiqueta: rec.motorcycles

--- Documento de Referencia ID: 6824 ---
Etiqueta: sci.space

Top 5 Documentos Más Similares:
1. ID: 11125 | Sim. Coseno: 0.2071 | Etiqueta: sci.space
2. ID: 8196 | Sim. Coseno: 0.1982 | Etiqueta: talk.religion.misc
3. ID: 784 | Sim. Coseno: 0.1872 | Etiqueta: alt.atheism
4. ID: 3400 | Sim. Coseno: 0.1775 | Etiqueta: soc.religion.christian
5. ID: 3707 | Sim. Coseno: 0.1747 | Etiqueta: sci.space

--- Documento de Referencia ID: 6792 ---
Etiqueta: talk.politics.guns

Top 5 Documentos Más Similares:
1. ID: 228 | Sim. Coseno: 0.3197 | Etiqueta: talk.politics.guns
2. ID: 2885 | Sim. Coseno: 0.2338 | Etique

La vectorización TF-IDF ha logrado un rendimiento coherente al identificar documentos vecinos.

Validación de Temas Relacionados: La matriz captura con éxito las macro-categorías, como se ve en la fuerte similaridad entre rec.motorcycles y rec.autos. Esto valida que el proceso de tokenización y el cálculo de la frecuencia de términos funcionan para agrupar temas afines.

Baja Densidad de la Similaridad: Los valores de Similaridad del Coseno son generalmente bajos (máximo 0.3197), lo que indica que los documentos, incluso los más similares, no son copias idénticas y que el vocabulario de 20 Newsgroups es muy amplio y disperso.

Captura de Contenido Subyacente: El caso del documento ID 6824 (sci.space) con fuerte traslape hacia temas religiosos, demuestra que el TF-IDF no solo captura la etiqueta principal, sino el vocabulario y el subtema específico que se discute dentro del documento, lo cual es la principal fortaleza de este tipo de vectorización.

---

**2**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [17]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 1. Transponer la matriz (Obtenemos la matriz Término-Documento)
# Cada fila de X_word_vectors es ahora el vector TF-IDF de una palabra.
X_word_vectors = X_train.transpose()

# 2. Obtener el vocabulario (el mapeo de índice a palabra)
vocab = tfidfvect.get_feature_names_out()

# 3. Definir las 5 palabras a estudiar (Ejemplo de palabras)
# NOTA: Debes verificar que estas palabras existan en tu vocabulario 'vocab'
palabras_a_estudiar = ['windows', 'gun', 'god', 'space', 'engine']

# 4. Obtener los índices de las palabras en el vocabulario
indices_palabras = []
for palabra in palabras_a_estudiar:
    if palabra in tfidfvect.vocabulary_:
        indices_palabras.append(tfidfvect.vocabulary_[palabra])
    else:
        print(f"Advertencia: La palabra '{palabra}' no se encontró en el vocabulario y será ignorada.")

# 5. Calcular la similaridad del coseno y rankear los resultados
print("\n--- Análisis de Similaridad entre Palabras ---")

for idx_palabra_ref in indices_palabras:
    palabra_ref = vocab[idx_palabra_ref]

    # Extraer el vector de la palabra de referencia
    vector_palabra_ref = X_word_vectors[idx_palabra_ref]

    # Calcular la similaridad del coseno entre el vector de referencia y TODOS los demás vectores de palabras
    # Usamos .toarray() para asegurarnos de que el cálculo con la matriz dispersa funcione bien
    similaridades = cosine_similarity(vector_palabra_ref.toarray(), X_word_vectors).flatten()

    # Obtener el ranking de los índices más similares (de mayor a menor)
    indices_ranking = np.argsort(similaridades)[::-1]

    # Seleccionar el Top 5 (excluyendo el primer elemento, que es la palabra consigo misma)
    top_5_indices = indices_ranking[1:6]

    print(f"\nPalabra de Referencia: '{palabra_ref.upper()}'")
    print("-"*60)

    # Mostrar resultados
    for i, idx_similar in enumerate(top_5_indices):
        score = similaridades[idx_similar]
        palabra_similar = vocab[idx_similar]
        print(f"  {i+1}. '{palabra_similar}' | Sim. Coseno: {score:.4f}")


--- Análisis de Similaridad entre Palabras ---

Palabra de Referencia: 'WINDOWS'
------------------------------------------------------------
  1. 'dos' | Sim. Coseno: 0.3084
  2. 'ms' | Sim. Coseno: 0.2249
  3. 'microsoft' | Sim. Coseno: 0.2074
  4. 'nt' | Sim. Coseno: 0.1973
  5. 'file' | Sim. Coseno: 0.1926

Palabra de Referencia: 'GUN'
------------------------------------------------------------
  1. 'guns' | Sim. Coseno: 0.3752
  2. 'handgun' | Sim. Coseno: 0.2447
  3. 'crime' | Sim. Coseno: 0.2435
  4. 'firearms' | Sim. Coseno: 0.2378
  5. 'homicides' | Sim. Coseno: 0.2310

Palabra de Referencia: 'GOD'
------------------------------------------------------------
  1. 'jesus' | Sim. Coseno: 0.2768
  2. 'bible' | Sim. Coseno: 0.2675
  3. 'christ' | Sim. Coseno: 0.2674
  4. 'faith' | Sim. Coseno: 0.2546
  5. 'existence' | Sim. Coseno: 0.2492

Palabra de Referencia: 'SPACE'
------------------------------------------------------------
  1. 'nasa' | Sim. Coseno: 0.3178
  2. 'shuttle' 

La vectorización de palabras basada en la transposición de la matriz TF-IDF resultó ser altamente efectiva para capturar las relaciones semánticas y los contextos temáticos. Los términos técnicos (WINDOWS, SPACE, GUN, GOD) mostraron una fuerte cohesión temática y un score de similaridad más alto (máx. 0.3752), mientras que la palabra más general (ENGINE) mostró una similaridad más baja y distribuida, reflejando la dispersión de su uso en contextos variados del corpus.

---
**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Definir el Pipeline
pipeline = Pipeline([
    # Fijamos max_df=0.8 y min_df=5 aquí para reducir el espacio de búsqueda.
    ('tfidf', TfidfVectorizer(stop_words='english', max_df=0.8, min_df=5)),
    ('clf', MultinomialNB())
])

# 2. Definir el espacio de parámetros reducido
param_grid_optimizado = [
    { # Grid para MultinomialNB
        'clf': [MultinomialNB()],
        # Solo variamos ngram_range y alpha
        'tfidf__ngram_range': [(1, 1), (1, 2)],
        'clf__alpha': [0.1, 1.0] # Reducido a dos valores
    },
    { # Grid para ComplementNB
        'clf': [ComplementNB()],
        # Solo variamos ngram_range y alpha
        'tfidf__ngram_range': [(1, 1), (1, 2)],
        'clf__alpha': [0.1, 1.0] # Reducido a dos valores
    }
]

# Total de Entrenamientos Ahora: 2 Modelos * 2 n-grams * 2 alphas * 3 CV = 24 entrenamientos.

# 3. Instanciar y Entrenar GridSearchCV
grid_search_opt = GridSearchCV(
    pipeline,
    param_grid_optimizado,
    scoring='f1_macro',
    cv=3,
    verbose=2, # verbose=2 muestra el progreso de cada iteración
    n_jobs=-1
)

print("Iniciando Grid Search Optimizado (Solo 24 entrenamientos)...")
grid_search_opt.fit(newsgroups_train.data, newsgroups_train.target)
print("Grid Search Completado.")

Iniciando Grid Search Optimizado (Solo 24 entrenamientos)...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Grid Search Completado.


In [24]:
from sklearn.metrics import f1_score

# Mostrar el mejor resultado de la búsqueda
print("="*60)
print("RESULTADOS DE LA BÚSQUEDA EN EL CONJUNTO DE ENTRENAMIENTO (CV):")
print(f"Mejor F1-score macro en CV: {grid_search_opt.best_score_:.4f}")
print("Mejores parámetros:", grid_search_opt.best_params_)
print("="*60)

# Obtener el mejor modelo entrenado
best_model = grid_search_opt.best_estimator_

RESULTADOS DE LA BÚSQUEDA EN EL CONJUNTO DE ENTRENAMIENTO (CV):
Mejor F1-score macro en CV: 0.7253
Mejores parámetros: {'clf': ComplementNB(), 'clf__alpha': 1.0, 'tfidf__ngram_range': (1, 2)}


In [25]:
# Evaluar el rendimiento en el conjunto de datos de TEST
print("\nEVALUACIÓN FINAL EN EL CONJUNTO DE TEST:")

# 1. Realizar la predicción en los datos de test (texto original)
y_pred_test = best_model.predict(newsgroups_test.data)

# 2. Calcular el f1-score macro final
f1_final_test = f1_score(newsgroups_test.target, y_pred_test, average='macro')

print(f"El mejor F1-score macro final en Test es: {f1_final_test:.4f}")


EVALUACIÓN FINAL EN EL CONJUNTO DE TEST:
El mejor F1-score macro final en Test es: 0.6915


#Conclusión
La clasificación óptima se logró utilizando el modelo Complement Naïve Bayes con un suavizado de  alpha=1 y un vectorizador TF-IDF que incorpora bi-gramas (ngram_range=(1, 2)). El F1-score macro final de $0.6915$ en el conjunto de prueba confirma que la combinación de los parámetros de TfidfVectorizer y el uso de ComplementNB es la estrategia más efectiva para clasificar este corpus de texto.